Coursera IBM Capstone
==
## Segmenting and Clustering Neighborhoods in Toronto
### By: Ted Hartnell
In this assignment, you will be required to explore, segment, and cluster the neighborhoods in the city of Toronto. However, unlike New York, the neighborhood data is not readily available on the internet. What is interesting about the field of data science is that each project can be challenging in its unique way, so you need to learn to be agile and refine the skill to learn new libraries and tools quickly depending on the project.

For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. You will be required to scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe so that it is in a structured format like the New York dataset.

Once the data is in a structured format, you can replicate the analysis that we did to the New York City dataset to explore and cluster the neighborhoods in the city of Toronto.

Your submission will be a link to your Jupyter Notebook on your Github repository.

## Step 1: Load Dependencies

In [1]:
import pandas as pd
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

In [2]:
pip install wikipedia

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Import the Python Wikipedia Crawl library
import wikipedia
wikipedia.summary("List of postal codes of Canada: M", sentences=2)

'This is a list of postal codes in Canada where the first letter is M. Postal codes beginning with M are located within the city of Toronto in the province of Ontario. Only the first three characters are listed, corresponding to the Forward Sortation Area.'

In [4]:
# Import BeautifulSoup to parse the crawled HTML
try: 
    from BeautifulSoup import BeautifulSoup
except ImportError:
    from bs4 import BeautifulSoup

In [5]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


In [6]:
# Import the Geocoder Library to lookup Latitude and Longitude of Toronto Neighborhoods
import geocoder

In [7]:
# Import library to handle REST requests
import requests

In [8]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [9]:
# Import Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# Import k-means from clustering stage
from sklearn.cluster import KMeans

# Import Map Rendering Library
import folium

In [10]:
# Have the user manually enter the Foursquare Credentials so they are not stored in the raw Jupyter notebook
# Clear Cell Outputs after running!!!
FOURSQUARE_CLIENT_ID = input("[1/3] Enter your FourSquare Client ID:")
FOURSQUARE_CLIENT_SECRET = input("[2/3] Enter your FourSquare Client Secret:")
FOURSQUARE_VERSION = input("[3/3] Enter your FourSquare Version Number:")

## Step 2: Crawl Wikipedia Toronto Neighborhoods

In [11]:
# Get the link to the 'List of Neighborhoods in Toronto' Wikipedia Page and test it
wikipediaNeighborhoods = wikipedia.page("List_of_postal_codes_of_Canada:_M")
print( wikipediaNeighborhoods.title )
print( wikipediaNeighborhoods.url )

List of postal codes of Canada: M
https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M


In [12]:
# Get the HTML content from the 'List of Neighborhoods in Toronto' Wikipedia Page as a String
htmlNeighborhoods = wikipediaNeighborhoods.html()

# Check the HTML content was collected
print( htmlNeighborhoods[0:100] )

<div class="mw-parser-output"><p>This is a list of <a href="/wiki/Postal_codes_in_Canada" title="Pos


In [13]:
# Parse the HTML so it can be searched by BeautifulSoup
parsedNeighborhoods = BeautifulSoup(htmlNeighborhoods)

In [14]:
# Find the list of Neighborhoods at the bottom of the 'List of Neighborhoods in Toronto' Wikipedia Page
find_allNeighborhoods = parsedNeighborhoods.find('table', class_='wikitable').find_all('tr', class_='')
print('Found {} potential Neighborhoods in the HTML'.format( len(find_allNeighborhoods) ))

# Check that the first 5 neighborhoods contains useful data
find_allNeighborhoods[:5]

Found 289 potential Neighborhoods in the HTML


[<tr>
 <th>Postcode</th>
 <th>Borough</th>
 <th>Neighbourhood
 </th></tr>, <tr>
 <td>M1A</td>
 <td>Not assigned</td>
 <td>Not assigned
 </td></tr>, <tr>
 <td>M2A</td>
 <td>Not assigned</td>
 <td>Not assigned
 </td></tr>, <tr>
 <td>M3A</td>
 <td><a href="/wiki/North_York" title="North York">North York</a></td>
 <td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
 </td></tr>, <tr>
 <td>M4A</td>
 <td><a href="/wiki/North_York" title="North York">North York</a></td>
 <td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
 </td></tr>]

In [15]:
# Define the dataframe columns for the Neighborhoods
columnsNeighborhoods = ['PostalCode', 'Borough', 'Neighborhood', 'URL', 'Title', 'Latitude', 'Longitude'] 

# Instantiate the dataframe
neighborhoods = pd.DataFrame(columns=columnsNeighborhoods)

In [25]:
# Try reading in the neighborhoods from earlier - necessary as it takes a very long time to parse neighborhood data
neighborhoods = pd.read_csv("Toronto Neighborhoods 013 Tab-Delineated.csv", sep='\t')
neighborhoods.drop(neighborhoods.columns[0], axis=1, inplace=True) # Drop the ID column

In [26]:
# Convert crawled neighborhood HTML data into structured DataFrames
# Note that this routine can be run several times to retry latitude/longitude lookups as they very often fail

# Limit the number of neighborhoods that can be added each time this cell is run
MAX_NEIGHBORHOODS = 300
countNeighborhoods = 0

for row in find_allNeighborhoods:
    
    countNeighborhoods += 1
    if countNeighborhoods > MAX_NEIGHBORHOODS:
        break
    
    # Get the list of rows found within each crawled table fragment
    tag = row.findAll('td')

    # Extract the Postal Code
    try:
        postalcodeNeighborhood = tag[0].get_text().strip()
    except:
        print("Row {} doesn't have any Neighborhood information - Skip".format(countNeighborhoods))
        continue
    
    # Extract the Borough - if the Borough is 'Not assigned' then skip the row
    try:
        boroughNeighborhood = tag[1].get_text().strip()
    except:
        print("Row {} doesn't have a Borough - Skip".format(countNeighborhoods))
        continue

    if boroughNeighborhood == 'Not assigned':
        print("Row {} Borough is marked as 'Not Assigned' - Skip".format(countNeighborhoods))
        continue
        
    # Extract the Name of the Neighborhood and clean it up
    try:
        nameNeighborhood = tag[2].get_text().strip()
    except:
        print("Row {} doesn't have a Neighborhood - Skip".format(countNeighborhoods))
        continue

    nameNeighborhood = nameNeighborhood.replace(", Toronto","").replace(" (Toronto)","").replace(" (neighbourhood)","").replace(" (page does not exist)","")
    
    # If the Neighborhood Name is 'Not assigned' then use the Borough Name instead
    if nameNeighborhood == 'Not assigned':
        nameNeighborhood = boroughNeighborhood

    # Extract the URL for the Neighborhood (or Borough)
    try:
        urlNeighborhood = tag[2].a['href'].strip() # Try getting the URL for the Neighborhood
    except:
        try:
            urlNeighborhood = tag[1].a['href'].strip() # Else try getting the URL for the Borough
        except:
            urlNeighborhood = ''

    # Extract the Title for the Neighborhood - the Title often contains more information so is used for looking up the Latitude and Longitude
    try:
        titleNeighborhood = tag[2].a['title'].strip() # Try getting the Title for the Neighborhood
    except:
        try:
            titleNeighborhood = tag[1].a['title'].strip() # Else try getting the Title for the Borough
        except:
            titleNeighborhood = ''

    titleNeighborhood = titleNeighborhood.replace(", Toronto","").replace(" (Toronto)","").replace(" (neighbourhood)","").replace(" (page does not exist)","")
    titleNeighborhood = titleNeighborhood + ', Toronto, Ontario' # Use the Neighborhood Title to find the latitude and longitude
            
    # Check that the Neighborhood Name being added to the DataFrame is unique - if the Name is not unique then it has already been added to the DataFrame (recall that this routine can be run many times in order to ensure all location data is loaded)
    if nameNeighborhood in neighborhoods["Neighborhood"].tolist():
        print('The Neighborhood of {} has already been loaded into the DataFrame'.format(nameNeighborhood))
        continue
    
    latitudeNeighborhood = ''
    longitudeNeighborhood = ''

    # (optional) Lookup the Latitude and Longitude of the Neighborhood
    if True:
        try:
            locationNeighborhood = geocoder.arcgis('{}, Toronto, Ontario'.format(nameNeighborhood))
            latitudeNeighborhood = locationNeighborhood.latlng[0]
            longitudeNeighborhood = locationNeighborhood.latlng[1]
            print('{}: The geograpical coordinate of {} is {}, {}.'.format(postalcodeNeighborhood, titleNeighborhood, latitudeNeighborhood, longitudeNeighborhood))
        except:
            print('{}: The geograpical coordinates of {} failed'.format(postalcodeNeighborhood, titleNeighborhood))
            latitudeNeighborhood = ''
            longitudeNeighborhood = ''
            continue

    print('{}: Adding Neighborhood {}'.format(postalcodeNeighborhood, titleNeighborhood))
    neighborhoods = neighborhoods.append({
                                            'PostalCode': postalcodeNeighborhood,
                                            'Borough': boroughNeighborhood,
                                            'Neighborhood': nameNeighborhood,
                                            'URL': urlNeighborhood,
                                            'Title': titleNeighborhood,
                                            'Latitude': latitudeNeighborhood,
                                            'Longitude': longitudeNeighborhood
                                            }, ignore_index=True)

# Note that not all of the locations of the Neighborhood names from the Wikipedia page can be found by the geolocator
print('Finished adding {} Neighborhoods from {} crawled rows.'.format(len(neighborhoods), len(find_allNeighborhoods), ))

Row 1 doesn't have any Neighborhood information - Skip
Row 2 Borough is marked as 'Not Assigned' - Skip
Row 3 Borough is marked as 'Not Assigned' - Skip
The Neighborhood of Parkwoods has already been loaded into the DataFrame
The Neighborhood of Victoria Village has already been loaded into the DataFrame
The Neighborhood of Harbourfront has already been loaded into the DataFrame
The Neighborhood of Regent Park has already been loaded into the DataFrame
The Neighborhood of Lawrence Heights has already been loaded into the DataFrame
The Neighborhood of Lawrence Manor has already been loaded into the DataFrame
The Neighborhood of Queen's Park has already been loaded into the DataFrame
Row 11 Borough is marked as 'Not Assigned' - Skip
The Neighborhood of Islington Avenue has already been loaded into the DataFrame
The Neighborhood of Rouge has already been loaded into the DataFrame
The Neighborhood of Malvern has already been loaded into the DataFrame
Row 15 Borough is marked as 'Not Assign

In [27]:
# Check that the neighborhoods have been loaded correctly from the crawled results
print( neighborhoods.shape )
neighborhoods.head()

(209, 7)


,PostalCode,Borough,Neighborhood,URL,Title,Latitude,Longitude
0,M3A,North York,Parkwoods,/wiki/Parkwoods,"Parkwoods, Toronto, Ontario",43.686575,-79.409993
1,M4A,North York,Victoria Village,/wiki/Victoria_Village,"Victoria Village, Toronto, Ontario",43.731540,-79.314280
2,M5A,Downtown Toronto,Regent Park,/wiki/Regent_Park,"Regent Park, Toronto, Ontario",43.660690,-79.360310
3,M6A,North York,Lawrence Heights,/wiki/Lawrence_Heights,"Lawrence Heights, Toronto, Ontario",43.723570,-79.437110
4,M6A,North York,Lawrence Manor,/wiki/Lawrence_Manor,"Lawrence Manor, Toronto, Ontario",43.722920,-79.431310


In [28]:
# Group the Neighborhoods by Postal Code so that the Neighborhoods are concatenated
groupNeighborhoodNames = neighborhoods.groupby(['PostalCode','Borough'], as_index=True)['Neighborhood'].apply(', '.join).reset_index()

# Group the Neighborhoods by Postal Code so that the Location is averaged
groupNeighborhoodLocations = neighborhoods.groupby(['PostalCode','Borough'], as_index=True)['Latitude','Longitude'].mean()

# Merge together the final Grouped Neighborhoods
neighborhoods = pd.merge(groupNeighborhoodNames, groupNeighborhoodLocations, on=['PostalCode','Borough'], how='inner')

print( neighborhoods.shape )
neighborhoods.head(20)

(103, 5)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.808715,-79.197445
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.785203,-79.146583
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.765170,-79.191117
3,M1G,Scarborough,Woburn,43.767300,-79.228230
4,M1H,Scarborough,Cedarbrae,43.747728,-79.235174
5,M1J,Scarborough,Scarborough Village,43.738520,-79.216920
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.747869,-79.277956
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.713880,-79.287330
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.722914,-79.233510
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.694081,-79.263270


In [29]:
# Get the Location of the geographic center of Toronto to center the map
nameLocation = "Leaside, Toronto, Ontario" # The geographic center is not "Toronto, Canada"
locationToronto = geocoder.arcgis(nameLocation)
print('The geograpical coordinate of the city center at {} is {}, {}.'.format(nameLocation, locationToronto.latlng[0], locationToronto.latlng[1]))

The geograpical coordinate of the city center at Leaside, Toronto, Ontario is 43.70023937855272, -79.3510651247247.


In [30]:
# Create map of Toronto using {latitude, longitude} values and Neighborhood label
mapToronto = folium.Map(location=[locationToronto.latlng[0], locationToronto.latlng[1]], zoom_start=11)

# Add markers to Map
for latitude, longitude, postalcode, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['PostalCode'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '[' + postalcode +']: ' + neighborhood + ', ' + borough
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(mapToronto)  
    
mapToronto

### Toronto Static Map of Neighborhoods
![Toronto Static Map of Neighborhoods](Toronto_Neighborhood_Map_003.png "Toronto Static Map of Neighborhoods")

## The End.